# Extracción y descripción de los datos

Empiezo importando las librerías que voy a utilizar.

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

Continuo creando 3 funciones con las que voy a hacer los siguiente:
 - La primera divide un archivo audio de larga duración en pequeños audios de 10 segundos de duración, y los guarda en la carpeta que le he especificado.
 - La segunda accede a cada uno de esos audios de 10 segundos y saca las features características de dicho archivo, y en base a su nombre le asigna una categoría y otra.
 - La tercera simplemente me enseña las principales características y métricas de la base de datos.

Finalmente, guardo el el dataset en mi carpeta como un CSV.

Dado que la base de datos ya la tengo creada, este JN lo voy a aplicar sobre audios que luego voy a utilizar para predecir con mi modelo y ver si es lo suficientemente bueno.

In [4]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 2000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            #print("exporting", chunk_name)
            chunk.export('./audios/fourier_32/{}'.format(chunk_name), format="mp3")
            
decompose_files('./audios/audios_base/', glob('./audios/audios_base' + '/*'))

In [ ]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=8000, mono=True)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr, n_bands=4))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        D = np.abs(np.fft.fft(y, n=64)[:8000 // 2])
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, 
                         D[0], D[1], D[2], D[3], D[4], D[5], 
                         D[6], D[7], D[8], D[9], D[10], D[11], 
                         D[12], D[13], D[14], D[15], D[16], D[17], 
                         D[18], D[19], D[20], D[21], D[22], D[23],
                         D[24], D[25], D[26], D[27], D[28], D[29],
                         D[30], D[31], D[32], D[33], D[34], D[35],
                         D[36], D[37], D[38], D[39], D[40], D[41],
                         D[42], D[43], D[44], D[45], D[46], D[47],
                         D[48], D[49], D[50], D[51], D[52], D[53],
                         D[54], D[55], D[56], D[57], D[58], D[59],
                         D[60], D[61], D[62], D[63], momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
                                           'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
                                           'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
                                           'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
                                           'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
                                           'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
                                           'Fourier31', 'Fourier32', 'Fourier33', 'Fourier34', 'Fourier35', 
                                           'Fourier36', 'Fourier37', 'Fourier38', 'Fourier39', 'Fourier40',
                                           'Fourier41', 'Fourier42', 'Fourier43', 'Fourier44', 'Fourier45',
                                           'Fourier46', 'Fourier47', 'Fourier48', 'Fourier49', 'Fourier50',
                                           'Fourier51', 'Fourier52', 'Fourier53', 'Fourier54', 'Fourier55',
                                           'Fourier56', 'Fourier57', 'Fourier58', 'Fourier59', 'Fourier60',
                                           'Fourier61', 'Fourier62', 'Fourier63', 'Fourier64', 'momento'])

data = get_features('./audios/fourier_32/', glob('./audios/fourier_32' + '/*.wav'))

In [4]:
data.shape

(14611, 72)

En este paso intermedio, dado que hay una pequeña cantidad de audios cuyas features son todas cero, los elimino para no ensuciar el modelo.

In [5]:
data = data[data['Fourier5'] != 0].reset_index(drop=True)

In [6]:
def show_df_info(dataframe):
    display("The type of the dataframe is {}.".format(type(dataframe)))
    print("------------------------------------------------------------------------")
    display("The amount of entries are {}.".format(dataframe.size))
    print("------------------------------------------------------------------------")
    display("Its shape is {}.".format(dataframe.shape))
    print("------------------------------------------------------------------------")
    display("Its features are: {}.".format(list(dataframe)))
    print("------------------------------------------------------------------------")
    print("The data types of columns are: {}".format(dataframe.dtypes))
    print("------------------------------------------------------------------------")
    print("The counter of each class is: {}".format(dataframe.momento.value_counts()))
    print("------------------------------------------------------------------------")
    print('The amount of null values is: {}.'.format(dataframe.isna().sum()))
    display(dataframe.head())
    display(dataframe.describe())
    
show_df_info(data)

"The type of the dataframe is <class 'pandas.core.frame.DataFrame'>."

------------------------------------------------------------------------


'The amount of entries are 1030824.'

------------------------------------------------------------------------


'Its shape is (14317, 72).'

------------------------------------------------------------------------


"Its features are: ['mfcc', 'scem', 'scom', 'srom', 'sbwm', 'tempo', 'rmse', 'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10', 'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15', 'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20', 'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25', 'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30', 'Fourier31', 'Fourier32', 'Fourier33', 'Fourier34', 'Fourier35', 'Fourier36', 'Fourier37', 'Fourier38', 'Fourier39', 'Fourier40', 'Fourier41', 'Fourier42', 'Fourier43', 'Fourier44', 'Fourier45', 'Fourier46', 'Fourier47', 'Fourier48', 'Fourier49', 'Fourier50', 'Fourier51', 'Fourier52', 'Fourier53', 'Fourier54', 'Fourier55', 'Fourier56', 'Fourier57', 'Fourier58', 'Fourier59', 'Fourier60', 'Fourier61', 'Fourier62', 'Fourier63', 'Fourier64', 'momento']."

------------------------------------------------------------------------
The data types of columns are: mfcc         float64
scem         float64
scom         float64
srom         float64
sbwm         float64
tempo        float64
rmse         float64
Fourier1     float64
Fourier2     float64
Fourier3     float64
Fourier4     float64
Fourier5     float64
Fourier6     float64
Fourier7     float64
Fourier8     float64
Fourier9     float64
Fourier10    float64
Fourier11    float64
Fourier12    float64
Fourier13    float64
Fourier14    float64
Fourier15    float64
Fourier16    float64
Fourier17    float64
Fourier18    float64
Fourier19    float64
Fourier20    float64
Fourier21    float64
Fourier22    float64
Fourier23    float64
              ...   
Fourier36    float64
Fourier37    float64
Fourier38    float64
Fourier39    float64
Fourier40    float64
Fourier41    float64
Fourier42    float64
Fourier43    float64
Fourier44    float64
Fourier45    float64
Fourier46    float64
Fourier47    f

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier56,Fourier57,Fourier58,Fourier59,Fourier60,Fourier61,Fourier62,Fourier63,Fourier64,momento
0,-19.602940,1016.689071,28.116497,1344.604492,526.793735,117.187500,0.082146,0.018555,0.015966,0.020310,...,0.036940,0.063189,0.284137,0.063383,0.102563,0.036719,0.019779,0.020310,0.015966,Timbre
1,-23.112227,1361.218925,25.115322,2646.606445,1025.292241,117.187500,0.055449,0.221948,0.244370,0.244570,...,0.036473,0.084608,0.108805,0.122175,0.109803,0.547955,0.635384,0.244570,0.244370,Cena
2,4.153023,662.289054,21.617422,1497.436523,832.083155,156.250000,0.240111,3.205282,12.572833,1.982500,...,0.472631,0.274115,0.177703,0.353226,0.455353,0.413510,0.272553,1.982500,12.572833,Lavadora
3,-15.240392,1539.737568,22.446293,2865.844727,1002.431942,133.928571,0.113920,0.131522,0.614816,0.363291,...,2.669167,0.669400,1.199691,1.602963,3.889082,0.397376,0.761557,0.363291,0.614816,Secador_pelo
4,6.963927,582.127562,19.931546,1348.144531,798.607193,133.928571,0.212797,5.507216,1.900305,0.823831,...,0.154926,0.249078,0.624590,0.252448,0.586256,0.691590,1.555114,0.823831,1.900305,Aire_acondicionado


,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier55,Fourier56,Fourier57,Fourier58,Fourier59,Fourier60,Fourier61,Fourier62,Fourier63,Fourier64
count,14317.000000,14317.000000,14317.000000,14317.000000,14317.000000,14317.000000,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,...,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04,1.431700e+04
mean,-8.975674,1254.326701,22.267985,2249.474068,892.656208,130.348634,8.845316e-02,1.073632e+00,1.473570e+00,1.051841e+00,...,3.253452e-01,3.668938e-01,3.769518e-01,5.267566e-01,5.206251e-01,6.607482e-01,8.611619e-01,9.009214e-01,1.051841e+00,1.473570e+00
std,10.089034,554.559052,3.257823,810.162213,197.184575,19.886981,8.053672e-02,2.340416e+00,2.559341e+00,1.730712e+00,...,4.988586e-01,6.135525e-01,6.154636e-01,8.838577e-01,6.854161e-01,9.863301e-01,2.271212e+00,1.672453e+00,1.730712e+00,2.559341e+00
min,-56.459336,168.275230,1.698515,291.992188,101.931160,37.500000,5.919943e-08,2.800062e-12,2.806961e-12,2.821882e-12,...,3.139653e-12,3.070597e-12,3.003386e-12,2.937453e-12,2.884319e-12,2.853604e-12,2.841517e-12,2.834276e-12,2.821882e-12,2.806961e-12
25%,-16.455608,798.831594,19.928018,1515.258789,805.097363,117.187500,1.969259e-02,4.268549e-02,7.116426e-02,6.826373e-02,...,4.051603e-02,4.038269e-02,4.179440e-02,4.158474e-02,4.357287e-02,4.706779e-02,4.959087e-02,5.670576e-02,6.826373e-02,7.116426e-02
50%,-7.940215,1281.012813,21.832554,2380.615234,927.216385,133.928571,6.708072e-02,2.037759e-01,3.259047e-01,3.398935e-01,...,1.507105e-01,1.619576e-01,1.833593e-01,2.105103e-01,2.593778e-01,2.845868e-01,2.847706e-01,2.853103e-01,3.398935e-01,3.259047e-01
75%,-3.018813,1738.787047,23.659923,2838.378906,1038.395741,133.928571,1.351235e-01,8.272752e-01,1.706695e+00,1.259900e+00,...,3.903347e-01,4.485462e-01,4.874474e-01,6.199973e-01,7.520742e-01,8.248828e-01,9.101153e-01,1.004417e+00,1.259900e+00,1.706695e+00
max,8.775053,2707.461116,51.275412,3609.375000,1299.501746,312.500000,5.108476e-01,2.631798e+01,2.021490e+01,1.647404e+01,...,6.556945e+00,7.642108e+00,8.626682e+00,1.038958e+01,7.717574e+00,7.864245e+00,2.482451e+01,1.376832e+01,1.647404e+01,2.021490e+01


In [7]:
data.to_csv('data_fourier_64.csv', index=False)